# Trabalho Prático de Temas de Álgebra 2

### Hugo Rocha, PG52250, MMC 
### Simão Quintela, PG52257, MMC  

**Enunciado:**


**Considere uma matriz A de característica máxima, 4x3, e b um vector 4x1, aleatórios. Assuma que car(A) < car[A|b], ou de forma equivalente, que det( [A|b] ) != 0.
Use reflexões elementares ou rotações de Givens para encontrar uma base ortonormada do espaço das colunas de A.
Usando a base ortonormada obtida, encontre a solução no sentido dos mínimos quadrados de Ax=b.**

##### Selection of matrix A with maximum rank (4x3) and vector b (4x1) such that:

$$det( [A|b] ) \neq 0 $$

In [151]:
A = random_matrix(ZZ,4,3)
b = random_matrix(ZZ,4,1)
detAb = 0
rankA = 0

while (detAb == 0 or rankA != 3):
    A = random_matrix(ZZ,4,3)
    b = random_matrix(ZZ,4,1)
    detAb = det(block_matrix(1,2,[A, b]))
    rankA = A.rank()


print("Matrix A:")
pretty_print(A)
print(f"rank(A) = {rankA}")
print("\nVector b:")
pretty_print(b)
print(f"\ndet([A|b]) = {detAb}")


Matrix A:


[ -1   4  12]
[-27   1  -1]
[ -2   1   0]
[  5  -7   0]

rank(A) = 3

Vector b:


[-1]
[-1]
[-1]
[ 2]


det([A|b]) = 1492


##### QR factorization algorithm of a matrix using Givens Rotations:

In [152]:
def copy(to_copy):
    l = []
    for line in to_copy:
        l.append(list(line))
    return matrix(l)


def prod_matrix(L):
    
    result = L[0]
    for i in range(1,len(L)):
        result *= L[i]
    return result
    
    
def rot_givens_fact_QR(A):
    n = A.nrows()
    m = A.ncols()
    R = copy(A)
    In = identity_matrix(RR, n)
    Plist = []
    
    
    for i in range(0,m):
        for j in range(i+1,n):
            cos = R[i,i] / sqrt( (R[i,i]^2) + (R[j,i]^2) )
            sin = R[j,i] / sqrt( (R[i,i]^2) + (R[j,i]^2) )
            P = copy(In)
            P[i,i] = P[j,j] = cos
            P[j,i] = -sin
            P[i,j] = sin
            R = P*R
            Plist.append(P.transpose())
    
    Q = prod_matrix(Plist)
    
    return Q, R

In [153]:
Q, R = rot_givens_fact_QR(A)

#####  Visualization of matrices Q and R using the previous algorithm:

In [154]:
print("Matrix Q:")
pretty_print(Q)
print("\nMatrix R:")
pretty_print(R)


Matrix Q:


[-0.0362977004320314   0.501055413965127   0.864644129681155 0.00405932106395537]
[ -0.980037911664848  -0.181818288692871  0.0639918510152765  0.0487118527674649]
[-0.0725954008640628   0.105174367554001 -0.0593474334395407  -0.990023303931348]
[  0.181488502160157  -0.839537929126878   0.494745848042908  -0.132153452415437]


Matrix R:


[     27.5499546279118     -2.46824362937813     0.544465506480471]
[-2.59639561098177e-17      7.80434323860978      6.19448325627439]
[ 1.47145973873341e-17 -2.86510762222211e-17      10.3117377051586]
[ 2.20029907474346e-16 -1.07261677571194e-16     0.000000000000000]

**By obtaining the matrix Q, we have an orthonormed basis of the column space of A.**

**We have:**

$$ A = QR $$ 

**We want to calculate x such that:**

$$ Ax = b $$

**I.e.:** 

$$ QRx = b $$

**Since Q is an orthogonal matrix, that is, its inverse is its transpose, when multiplying both sides of the equation, on the left, by the transpose of Q, we obtain:**

$$ Rx = Q^T b $$

**As R is an upper triangular matrix, solving this equation in order to x, it becomes relatively simple because it is equivalent to solving a system of linear equations by the process of substitution.**

In [155]:
right_side =  (Q.transpose())*b

var('x1', 'x2', 'x3')
x = matrix([[x1], [x2], [x3]])



print("Right side of the system (multiplying the transpose of Q by b):")
pretty_print(right_side)
print("\nComplete System:")
pretty_print(R, x, right_side)


Right side of the system (multiplying the transpose of Q by b):


[ 1.45190801728126]
[-2.10348735108001]
[0.120203148828925]
[0.672945225269053]


Complete System:


[     27.5499546279118     -2.46824362937813     0.544465506480471]
[-2.59639561098177e-17      7.80434323860978      6.19448325627439]
[ 1.47145973873341e-17 -2.86510762222211e-17      10.3117377051586]
[ 2.20029907474346e-16 -1.07261677571194e-16     0.000000000000000] [x1]
[x2]
[x3] [ 1.45190801728126]
[-2.10348735108001]
[0.120203148828925]
[0.672945225269053]

**As the last row of the matrix R is composed only of zeros and the last value of the vector resulting from the multiplication**

$$ Q^Tb $$ 

**is different from 0, the system is impossible. However, it is possible to calculate an approximation using the substitution method for the remaining equations. This approximation aims to minimize the value of** 

$$\| Ax - b \|$$

**and is called the "least squares solution".**

##### Solving the remaining equations manually (a Python function could have been used directly):

In [156]:
x3 = right_side[2,0]/R[2,2]
x2 = (right_side[1,0] - R[1,2]*x3)/R[1,1]
x1 = (right_side[0,0] - R[0,2]*x3 - R[0,1]*x2)/R[0,0]
x = matrix([[x1],[x2],[x3]])

##### Visualization of the approximate solution obtained:

In [157]:
print("Approximate solution:")
pretty_print(x)

Approximate solution:


[0.0274941975771515]
[-0.278780149560005]
[0.0116569245907789]

##### Comparing the results of Ax and b: 

In [158]:
print("Result of Ax and b:")
pretty_print(A*x, b)

Result of Ax and b:


[ -1.00273170072782]
[ -1.03278040873387]
[-0.333768544714308]
[  2.08893203480579] [-1]
[-1]
[-1]
[ 2]

###### Visualization of the value of |Ax - b|:

In [159]:
import numpy as np

print("Value of ||Ax - b||:")
pretty_print(np.linalg.norm(A*x - b))

Value of ||Ax - b||:


0.6729452252690526

##### Comparing our solution with the solution that can be obtained directly with a function in Python:

In [160]:
print("Python's approximate solution:")
py_x = np.linalg.lstsq(A, b, rcond=None)[0]
pretty_print(matrix(py_x))

Python's approximate solution:


[ 0.02749419757715152]
[-0.27878014956000463]
[0.011656924590779001]

**As we can see our solution is the same as this one so it seems to be correct.**